# Upload sortings to dandi

author: laquitainesteeve@gmail.com

Execution time: < 1 min

Special hardware: on CPU, does not require GPU.

# Setup 

Activate demo virtual environment (envs/demo.yml)

```bash
python -m ipykernel install --user --name demo --display-name "demo"
```

In [ ]:
%%time 

# import python packages
import os
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import spikeinterface as si
from pynwb.file import NWBFile, Subject
from pynwb import NWBHDF5IO
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from neuroconv.tools.spikeinterface import add_recording_to_nwbfile, add_sorting_to_nwbfile
print("spikeinterface", spikeinterface.__version__)

# setup project
proj_path = "/home/steeve/steeve/epfl/code/spikebias/"
os.chdir(proj_path)

# setup pipeline parameters
dandiset_id = '001250'

# setup nwb files save paths
nwb_file_path = os.path.join(proj_path, "tmp/")


/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


spikeinterface 0.101.2
CPU times: user 3.41 s, sys: 1.77 s, total: 5.18 s
Wall time: 626 ms


## Custom functions

In [2]:
def load_sortings(sorting_folders, sorting_path):
    
    # load sorting extractors
    folder_paths = []
    sorting_list = []
    for folder_i in sorting_folders:
        folder_paths = os.path.join(sorting_path, folder_i)
        sorting_list.append(si.load_extractor(folder_paths))
    
    # unit-test
    assert len(sorting_list) == len(sorting_folders), "sorting files are missing"
    return sorting_list


def upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description, experiment_description):
    """
    IMPORTANT! For Dandi 0.69.3, file_path must follow DANDI convention: {/sub-<subject_id>/sub-<subject_id>_ses-<session_id>.nwb}
    """
    # create nwb files save path
    subject_path = os.path.join(nwb_file_path, f"{dandiset_id}/sub-{subject_id}/")
    os.makedirs(subject_path, exist_ok=True)

    # loop over SortingExtractor files
    for s_ix, sorter_i in enumerate(sorter_name):
        
        # file path
        file_path = os.path.join(subject_path, f"sub-{subject_id}_ses-{sorter_i}.nwb")

        # add nwb session metadata (child folder)
        nwbfile = NWBFile(
            session_description=session_description,
            identifier=str(uuid.uuid4()),
            session_start_time=datetime.now(tzlocal()),
            experimenter="Laquitaine, Steeve",
            lab="Blue Brain Project",
            institution="EPFL",
            experiment_description=experiment_description,
            session_id=sorter_i,
            related_publications="https://doi.org/10.1101/2024.12.04.626805",
            keywords=["Biophysical simulation", "dense extracellular recordings", "spike sorting"]
        )

        # add subject metadata (parent folder)
        nwbfile.subject = Subject(
            subject_id = subject_id,
            species = "Rattus norvegicus",
            age = "P14D",
            sex = "M",
            description = "Wistar Rat",
        )

        # add SortingExtractor to nwb
        add_sorting_to_nwbfile(sorting = sorting_list[s_ix], nwbfile = nwbfile)

        # write nwb file locally
        with NWBHDF5IO(path=file_path, mode="w") as io:
            io.write(nwbfile)

        # WARNINg: check file with nwbinspector <source_folder> --config dandi beforehand

    # upload nwb to dandiset
    os.system(
        f"""
        export DANDI_API_KEY='210e68743286d64e84743bd8980d5771ef82bf4d';
        cd {dandiset_id};
        dandi organize {subject_path} -f dry;
        dandi organize {subject_path};    
        dandi upload
        """
    )

    # delete temporary datasets
    os.system(f"rm -rf {nwb_file_path}")
    os.system(f"rm -rf {dandiset_id}")

## Initialize dandiset

In [3]:
%%time 

# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  
CPU times: user 79 μs, sys: 1.36 ms, total: 1.44 ms
Wall time: 1.7 s


2025-06-16 14:31:54,534 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.31.53Z-2294576.log


0

## Biophy npx spont

In [ ]:
%%time

# setup experiment parameters
subject_id = "001-fitted"
sorting_path = os.path.join(proj_path, "dataset/01_intermediate/sorting/0_silico/npx_spont/")
sorting_folders = ["SortingKS4", "SortingKS3", "SortingKS2_5", "SortingKS2", "SortingKS", "SortingHerdingspikes", "SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]
sorter_name = ["ks4", "ks3", "ks25", "ks2", "ks", "hs", "ks4_10m", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # full and 10 min sortings

# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

# load list of sorting extractors
sorting_list = load_sortings(sorting_folders, sorting_path)

# upload the sorting nwb files to dandi archive
upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.", experiment_description="Biophysical simulation of neuropixels in the spontaneous regime. Background noise and gain are fitted to Marques-Smith dataset.")

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-15 19:05:10,847 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks4.nwb -> sub-001-fitted/sub-001-fitted_ses-ks4.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks2_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks2.nwb -> sub-001-fitted/sub-001-fitted_ses-ks2.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks.nwb -> sub-001-fitted/sub-001-fitted_ses-ks.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks3_10m.nwb -> sub-001-fitted/sub-001-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-001-fitted/sub-001-fitted_ses-ks3.nwb -> sub-001-fitted/sub-00

2025-06-15 19:05:11,718 [    INFO] Organized 0 out of 12 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-15 19:05:11,718 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.05.10Z-1724577.log
2025-06-15 19:05:12,925 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.6s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.8s finished
2025-06-15 19:05:13,732 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-15 19:05:13,737 [    INFO] Organized 12 paths. Visit /home

PATH                                           SIZE     ERRORS PROGRESS STATUS                MESSAGE                        
dandiset.yaml                                  5.6 kB                   skipped               should be edited online        
sub-001-fitted/sub-001-fitted_ses-hs-10m.nwb   988.8 kB   0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-hs.nwb       3.1 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb   4.7 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks.nwb       14.7 MB    0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb  3.0 MB     0        100% done                  exists (older) - reuploading   
sub-001-fitted/sub-001-fitted_ses-ks2.nwb      9.9 MB     0        100% done                  exists (older) - reuploa

2025-06-15 19:05:25,192 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.05.14Z-1724963.log


## Biophy npx evoked

In [ ]:
%%time

# setup experiment parameters
subject_id = "002-fitted"
sorting_path = os.path.join(proj_path, "dataset/01_intermediate/sorting/0_silico/npx_evoked/")
sorting_folders = ["SortingKS4_full", "SortingKS3_full", "SortingKS2_5_full", "SortingKS2_full", "SortingKS_full", "SortingHerdingspikes_full", "SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]
sorter_name = ["ks4", "ks3", "ks25", "ks2", "ks", "hs", "ks4_10m", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # full and 10 min sortings

# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

# load list of sorting extractors
sorting_list = load_sortings(sorting_folders, sorting_path)

# upload the sorting nwb files to dandi archive
upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Biophysical simulation of neuropixels in the evoked regime. Background noise and gain are fitted to Marques-Smith dataset.", experiment_description="Biophysical simulation of neuropixels in the evoked regime. Background noise and gain are fitted to Marques-Smith dataset.")

/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-15 19:07:41,203 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.7s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.7s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.7s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks2_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks25_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-hs.nwb -> sub-002-fitted/sub-002-fitted_ses-hs.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks3_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks_10m.nwb -> sub-002-fitted/sub-002-fitted_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks.nwb -> sub-002-fitted/sub-002-fitted_ses-ks.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-002-fitted/sub-002-fitted_ses-ks4.nwb -> sub-002-fitte

2025-06-15 19:07:43,322 [    INFO] Loading metadata from 12 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.6s remaining:    1.8s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.6s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.8s finished
2025-06-15 19:07:44,120 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-15 19:07:44,125 [    INFO] Organized 12 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-15 19:07:44,125 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.07.42Z-1728586.log
2025-06-15 19:07:46,080 [    INFO] Found 25 files to consider


PATH                                           SIZE     ERRORS   PROGRESS STATUS                MESSAGE                  
dandiset.yaml                                  5.6 kB                     skipped               should be edited online  
sub-001-fitted/sub-001-fitted_ses-hs-10m.nwb                              ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-hs.nwb                                  ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks-10m.nwb                              ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks.nwb                                  ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks2-10m.nwb                             ERROR                 File not found           
sub-001-fitted/sub-001-fitted_ses-ks2.nwb                                 ERROR                 File not found           
sub-001-fitted/sub-001-f

2025-06-15 19:07:56,230 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.15-17.07.44Z-1728779.log
Error: File not found


## Marques-Smith

In [ ]:
%%time

# setup experiment parameters
subject_id = "vivo-marques-smith"
sorting_path = os.path.join(proj_path, "dataset/01_intermediate/sorting/1_vivo/marques/")
sorting_folders = ["SortingKS3", "SortingKS2_5", "SortingKS2", "SortingKS", "SortingHerdingspikes", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]
sorter_name = ["ks3", "ks25", "ks2", "ks", "hs", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # full and 10 min sortings

# download a local dandiset folder with yaml config
os.system("dandi download --download dandiset.yaml DANDI:001250")

# load list of sorting extractors
sorting_list = load_sortings(sorting_folders, sorting_path)

# upload the sorting nwb files to dandi archive
upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Marques-Smith neuropixels in the spontaneous regime.", experiment_description="Marques-Smith neuropixels in the spontaneous regime.")

PATH                 SIZE DONE    DONE% CHECKSUM STATUS    MESSAGE    
001250/dandiset.yaml                             skipped   no change  
Summary:                  0 Bytes                1 skipped 1 no change
                          <0.00%                                      


2025-06-16 15:13:51,406 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.13.50Z-2322807.log
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.96.1 is recommended
  warnings.warn(


ValueError: /home/steeve/steeve/epfl/code/spikebias/dataset/01_intermediate/sorting/1_vivo/marques/SortingKS4_10m is not a file or a folder. It should point to either a json, pickle file or a folder that is the result of extractor.save(...)

## Biophy dense probes 1-3

In [17]:
%%time
# upload sortings for each Horvath probe depth

subject = ["003", "004", "005"]
depths = [1,2,3]
sorting_folders = ["SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]
sorter_name = ["ks4_10m", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # 10 min sortings

for d_ix, depth in enumerate(depths):

    # setup experiment parameters
    subject_id = f"{subject[d_ix]}-fitted"
    sorting_path = os.path.join(proj_path, f"dataset/01_intermediate/sorting/0_silico/dense_probe{depth}/")

    # download a local dandiset folder with yaml config
    os.system("dandi download --download dandiset.yaml DANDI:001250")

    # load list of sorting extractors
    sorting_list = load_sortings(sorting_folders, sorting_path)

    # upload the sorting nwb files to dandi archive
    upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Biophysical simulation of dense probe {depth} in the spontaneous regime. Background noise and gain are fitted to Horvath depth {depth} dataset.", experiment_description="Biophysical simulation of dense probe {depth} in the spontaneous regime. Background noise and gain are fitted to Horvath depth {depth} dataset.")

PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


2025-06-16 15:02:02,616 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.02.01Z-2315662.log
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-16 15:02:06,057 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.7s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.7s 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-ks3_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-ks2_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-hs_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-hs-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-ks4_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-ks4-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-ks25_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-003-fitted/sub-003-fitted_ses-ks_10m.nwb -> sub-003-fitted/sub-003-fitted_ses-ks-10m.nwb


2025-06-16 15:02:07,954 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s finished
2025-06-16 15:02:08,593 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-16 15:02:08,596 [    INFO] Organized 6 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-16 15:02:08,596 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.02.07Z-2315887.log
2025-06-16 15:02:10,529 [    INFO] Found 7 files to consider


PATH                                           SIZE     ERRORS   PROGRESS STATUS                MESSAGE                       
dandiset.yaml                                  5.6 kB                     skipped               should be edited online       
sub-003-fitted/sub-003-fitted_ses-hs-10m.nwb   539.3 kB   0          100% done                  exists (older) - reuploading  
sub-003-fitted/sub-003-fitted_ses-ks-10m.nwb   1.9 MB     0          100% done                  exists (older) - reuploading  
sub-003-fitted/sub-003-fitted_ses-ks2-10m.nwb  1.2 MB     0          100% done                  exists (older) - reuploading  
sub-003-fitted/sub-003-fitted_ses-ks25-10m.nwb 1.2 MB     0          100% done                  exists (older) - reuploading  
sub-003-fitted/sub-003-fitted_ses-ks3-10m.nwb  845.5 kB   0          100% done                  exists (older) - reuploading  
sub-003-fitted/sub-003-fitted_ses-ks4-10m.nwb  928.4 kB   0          100% done                  exists (older) 

...

2025-06-16 15:02:32,723 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.02.31Z-2316701.log
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-16 15:02:38,465 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.7s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.7s 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-ks2_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-ks25_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-ks4_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-ks4-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-ks3_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-ks3-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-ks_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-005-fitted/sub-005-fitted_ses-hs_10m.nwb -> sub-005-fitted/sub-005-fitted_ses-hs-10m.nwb


2025-06-16 15:02:40,346 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s finished
2025-06-16 15:02:40,977 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-16 15:02:40,981 [    INFO] Organized 6 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-16 15:02:40,981 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.02.39Z-2316925.log
2025-06-16 15:02:42,887 [    INFO] Found 7 files to consider


PATH                                           SIZE     ERRORS   PROGRESS STATUS                MESSAGE                       
dandiset.yaml                                  5.6 kB                     skipped               should be edited online       
sub-005-fitted/sub-005-fitted_ses-hs-10m.nwb   566.2 kB   0          100% done                  exists (older) - reuploading  
sub-005-fitted/sub-005-fitted_ses-ks-10m.nwb   3.1 MB     0          100% done                  exists (older) - reuploading  
sub-005-fitted/sub-005-fitted_ses-ks2-10m.nwb  2.0 MB     0          100% done                  exists (older) - reuploading  
sub-005-fitted/sub-005-fitted_ses-ks25-10m.nwb 1.9 MB     0          100% done                  exists (older) - reuploading  
sub-005-fitted/sub-005-fitted_ses-ks3-10m.nwb  3.1 MB     0          100% done                  exists (older) - reuploading  
sub-005-fitted/sub-005-fitted_ses-ks4-10m.nwb  3.4 MB     0          100% done                  exists (older) 

2025-06-16 15:04:59,720 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-13.02.41Z-2317081.log


## Horvath depths 1-3

In [ ]:
%%time

# upload sortings for each Horvath probe depth
depths = [1,2,3]
sorting_folders = ["SortingKS4_10m", "SortingKS3_10m", "SortingKS2_5_10m", "SortingKS2_10m", "SortingKS_10m", "SortingHerdingspikes_10m"]
sorter_name = ["ks4_10m", "ks3_10m", "ks25_10m", "ks2_10m", "ks_10m", "hs_10m"] # 10 min sortings

for depth in depths:

    # setup experiment parameters
    subject_id = f"vivo-horvath-depth-{depth}"
    sorting_path = os.path.join(proj_path, f"dataset/01_intermediate/sorting/1_vivo/horvath/probe_{depth}/")

    # download a local dandiset folder with yaml config
    os.system("dandi download --download dandiset.yaml DANDI:001250")

    # load list of sorting extractors
    sorting_list = load_sortings(sorting_folders, sorting_path)

    # upload the sorting nwb files to dandi archive
    upload_sorting_nwb_to_dandi(sorting_list, dandiset_id, subject_id, nwb_file_path, session_description = "Horvath dense probe depth {depth} in the spontaneous regime.", experiment_description="Horvath dense probe depth {depth} in the spontaneous regime.")

PATH                 SIZE DONE    DONE% CHECKSUM STATUS MESSAGE   
001250/dandiset.yaml                             done   updated   
Summary:                  0 Bytes                1 done 1 updated 
                          <0.00%                                  


2025-06-16 14:55:52,760 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.55.51Z-2310928.log
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-16 14:55:56,960 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.7s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.7s 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks_10m.nwb -> sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks25_10m.nwb -> sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks4_10m.nwb -> sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks4-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-hs_10m.nwb -> sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-hs-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks2_10m.nwb -> sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tm

2025-06-16 14:55:58,848 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.7s finished
2025-06-16 14:55:59,526 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-16 14:55:59,531 [    INFO] Organized 6 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-16 14:55:59,531 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.55.58Z-2311132.log
2025-06-16 14:56:01,438 [    INFO] Found 7 files to consider


PATH                                                               SIZE     ERRORS   PROGRESS STATUS                MESSAGE                       
dandiset.yaml                                                      5.6 kB                     skipped               should be edited online       
sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-hs-10m.nwb   518.4 kB   0          100% done                  exists (older) - reuploading  
sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks-10m.nwb   2.8 MB     0          100% done                  exists (older) - reuploading  
sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks2-10m.nwb  1.6 MB     0          100% done                  exists (older) - reuploading  
sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks25-10m.nwb 1.6 MB     0          100% done                  exists (older) - reuploading  
sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ses-ks3-10m.nwb  1.5 MB     0          100% done                  ex

...

2025-06-16 14:56:25,945 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.56.25Z-2311971.log
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.6 is recommended
  warnings.warn(
/home/steeve/steeve/epfl/code/spikebias/envs/demo/lib/python3.10/site-packages/spikeinterface/core/base.py:1129: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.100.5 is recommended
  warnings.warn(
2025-06-16 14:56:30,856 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.7s remaining:    1.4s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.7s 

DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks25_10m.nwb -> sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks25-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks2_10m.nwb -> sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks2-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks_10m.nwb -> sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks4_10m.nwb -> sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks4-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tmp/001250/sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-hs_10m.nwb -> sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-hs-10m.nwb
DRY: /home/steeve/steeve/epfl/code/spikebias/tm

2025-06-16 14:56:32,822 [    INFO] Loading metadata from 6 files
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:    0.6s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s finished
2025-06-16 14:56:33,450 [    INFO] Symlink support autodetected; setting files_mode='symlink'
2025-06-16 14:56:33,455 [    INFO] Organized 6 paths. Visit /home/steeve/steeve/epfl/code/spikebias/001250/
2025-06-16 14:56:33,455 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.56.32Z-2312181.log
2025-06-16 14:56:35,371 [    INFO] Found 7 files to consider


PATH                                                               SIZE     ERRORS   PROGRESS STATUS                MESSAGE                  
dandiset.yaml                                                      5.6 kB                     skipped               should be edited online  
sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-hs-10m.nwb   478.9 kB   0          100% done                                           
sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks-10m.nwb   3.4 MB     0          100% done                                           
sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks2-10m.nwb  2.1 MB     0          100% done                                           
sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks25-10m.nwb 2.0 MB     0          100% done                                           
sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ses-ks3-10m.nwb  2.0 MB     0          100% done                                           
sub-vi

2025-06-16 14:56:40,704 [    INFO] Logs saved in /home/steeve/.local/state/dandi-cli/log/2025.06.16-12.56.34Z-2312349.log
